In [1]:
import json
import polars as pl

def load_item_parquet(file_path: str) -> pl.DataFrame:
    return pl.read_parquet('data/table/item_data.parquet')


df_item = load_item_parquet('data/table/item_data.parquet')

In [2]:
def write_csv(df: pl.DataFrame, file_path: str) -> None:
    df.write_csv(file_path)
write_csv(df_item, 'data/table/item_data.csv')

In [3]:
#group ngày tạo item theo tháng và năm
df_item_not_valid = df_item.filter(pl.col('created_date_year')>= 2025, pl.col('created_date_month')>=2)
df_item_not_valid.shape

(1335, 25)

In [4]:
purchase_data = pl.read_parquet('data/table/purchase_new.parquet')
purchase_data.head()

timestamp,user_id,item_id,event_value,price,date_key,quantity,customer_id,channel,payment,location,discount,created_year,created_month,created_day,created_weekday,season_purchase
i64,str,str,"decimal[38,4]","decimal[38,4]",i32,i32,i32,str,str,i32,"decimal[38,4]",i32,i8,i8,i8,str
1735064221,"""ca12702ddf55acaa9fb767e10faaa6…","""7115000000004""",1.0000,49000.0000,20241224,1,5254214,"""In-Store""","""VietQR""",656,0.0000,2024,12,24,2,"""Winter"""
1735068481,"""54314ce86269f4849f9a4c3fc85d94…","""0029130000030""",1.0000,69000.0000,20241224,1,7573232,"""In-Store""","""Tiền mặt""",143,0.0000,2024,12,24,2,"""Winter"""
1735069843,"""4799de4022b58689b2b9fb37122479…","""3496000000053""",2.0000,75000.0000,20241224,2,8187418,"""In-Store""","""MoMo""",213,0.0000,2024,12,24,2,"""Winter"""
1735069779,"""4799de4022b58689b2b9fb37122479…","""2700000000002""",2.0000,58500.0000,20241224,2,8187418,"""In-Store""","""MoMo""",213,13000.0000,2024,12,24,2,"""Winter"""
1735379373,"""3b2423f730815273dfd6f6220b0c5c…","""0029110000036""",1.0000,89000.0000,20241228,1,6931560,"""Android""","""MoMo""",590,10000.0000,2024,12,28,6,"""Winter"""


In [5]:
# Set options để hiển thị đầy đủ
pl.Config.set_tbl_rows(-1)  # Hiển thị tất cả các dòng
pl.Config.set_tbl_cols(-1)  # Hiển thị tất cả các cột
pl.Config.set_fmt_str_lengths(100)  # Độ dài string

# Lọc purchase_data để lấy các giao dịch hợp lệ
purchase_filtered = purchase_data

# Join với item data để lấy thông tin năm tạo item
purchase_with_item = purchase_filtered.join(
    df_item.select(['item_id', 'created_date_year']),
    on='item_id',
    how='inner'
)

# Thống kê theo tháng mua và năm tạo item
stats = purchase_with_item.group_by(['created_year', 'created_month', 'created_date_year']).agg([
    pl.count().alias('total_purchases'),
    pl.col('item_id').n_unique().alias('unique_items')
]).sort(['created_year', 'created_month', 'created_date_year'])

print("Thống kê mua hàng theo tháng và năm tạo item:")
print(stats)

# Pivot để dễ nhìn hơn
pivot_stats = stats.pivot(
    index=['created_year', 'created_month'],
    columns='created_date_year',
    values='total_purchases'
).sort(['created_year', 'created_month'])

print("\nBảng pivot (dễ đọc hơn):")
print(pivot_stats)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17616\1230911114.py:18: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('total_purchases'),


Thống kê mua hàng theo tháng và năm tạo item:
shape: (183, 5)
┌──────────────┬───────────────┬───────────────────┬─────────────────┬──────────────┐
│ created_year ┆ created_month ┆ created_date_year ┆ total_purchases ┆ unique_items │
│ ---          ┆ ---           ┆ ---               ┆ ---             ┆ ---          │
│ i32          ┆ i8            ┆ i32               ┆ u32             ┆ u32          │
╞══════════════╪═══════════════╪═══════════════════╪═════════════════╪══════════════╡
│ 2024         ┆ 1             ┆ 2011              ┆ 8019            ┆ 14           │
│ 2024         ┆ 1             ┆ 2012              ┆ 2488            ┆ 5            │
│ 2024         ┆ 1             ┆ 2013              ┆ 25972           ┆ 30           │
│ 2024         ┆ 1             ┆ 2014              ┆ 36305           ┆ 55           │
│ 2024         ┆ 1             ┆ 2015              ┆ 66670           ┆ 76           │
│ 2024         ┆ 1             ┆ 2016              ┆ 80617           ┆ 170    

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17616\1230911114.py:26: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  pivot_stats = stats.pivot(


In [6]:
pivot_stats

created_year,created_month,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
i32,i8,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
2024,1,8019,2488,25972,36305,66670,80617,121114,193353,184049,352214,598787,703480,529133,446,null
2024,2,8087,2790,27831,36659,71441,95233,131948,231670,195145,374751,608052,701249,583082,11449,null
2024,3,7064,1950,25850,35722,62654,81659,121131,193234,191021,364829,580999,636297,567522,36654,null
2024,4,6238,2278,24077,31794,75147,73127,114334,177678,180311,333715,546025,612889,551144,71950,null
2024,5,6496,2549,25371,32075,55158,82329,119343,190534,183454,354233,566376,648242,576072,188994,null
2024,6,6138,2216,24049,30468,55669,80645,118459,181336,171359,344226,552333,626718,582614,205935,null
2024,7,6412,2212,23052,31176,56777,68628,123013,181146,170331,346086,550896,614783,605166,212435,null
2024,8,6098,1865,21608,29791,58995,58154,113316,168602,177166,340696,543350,605906,597719,295724,null
2024,9,6143,2107,22267,29963,54296,54648,104399,155015,162922,327531,499791,576945,564577,351670,null


# Phân Tích: Repurchase Strategy

Kiểm tra xem có bao nhiêu % users MUA LẠI items trong dataset

In [7]:
# Phân tích repurchase behavior
repurchase_analysis = (
    purchase_data
    .group_by(['customer_id', 'item_id'])
    .agg([
        pl.len().alias('purchase_count'),
        pl.col('timestamp').min().alias('first_purchase'),
        pl.col('timestamp').max().alias('last_purchase')
    ])
    .with_columns([
        (pl.col('last_purchase') - pl.col('first_purchase')).dt.total_days().alias('days_between_purchases')
    ])
)

# Users đã mua lại items (purchase_count > 1)
repurchasers = repurchase_analysis.filter(pl.col('purchase_count') > 1)

print(f"📊 Repurchase Statistics:")
print(f"Total unique (user, item) pairs: {repurchase_analysis.shape[0]:,}")
print(f"Repurchased pairs: {repurchasers.shape[0]:,}")
print(f"Repurchase rate: {repurchasers.shape[0] / repurchase_analysis.shape[0] * 100:.2f}%")
print(f"\nAverage days between repurchases: {repurchasers['days_between_purchases'].mean():.1f}")
print(f"Median days between repurchases: {repurchasers['days_between_purchases'].median():.1f}")

# Top repurchased items
top_repurchased = (
    repurchasers
    .group_by('item_id')
    .agg([
        pl.len().alias('num_repurchasers'),
        pl.col('purchase_count').mean().alias('avg_purchases'),
        pl.col('days_between_purchases').mean().alias('avg_days_between')
    ])
    .sort('num_repurchasers', descending=True)
    .head(10)
)

print("\n🔥 Top 10 Most Repurchased Items:")
print(top_repurchased)

SchemaError: invalid series dtype: expected `Duration`, got `i64` for series with name `last_purchase`